# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298384367491                   -0.85    5.0
  2   -8.300173510932       -2.75       -1.26    1.0
  3   -8.300449191727       -3.56       -1.89    3.4
  4   -8.300462107478       -4.89       -2.78    1.4
  5   -8.300464285313       -5.66       -3.07    3.4
  6   -8.300464454640       -6.77       -3.25    1.0
  7   -8.300464546293       -7.04       -3.39    1.4
  8   -8.300464595000       -7.31       -3.52    1.1
  9   -8.300464629760       -7.46       -3.70    1.0
 10   -8.300464639497       -8.01       -3.86    1.1
 11   -8.300464643806       -8.37       -4.33    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67442590312                   -0.70    6.5
  2   -16.67870221961       -2.37       -1.14    1.1
  3   -16.67922558916       -3.28       -1.87    2.2
  4   -16.67928030080       -4.26       -2.76    4.5
  5   -16.67928598859       -5.25       -3.18    5.4
  6   -16.67928619489       -6.69       -3.47    3.4
  7   -16.67928622103       -7.58       -3.95    1.0
  8   -16.67928622400       -8.53       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.30793696309                   -0.56    6.4
  2   -33.33434592314       -1.58       -1.00    2.2
  3   -33.33597370404       -2.79       -1.73    7.9
  4   -33.33615501944       -3.74       -2.66    3.9
  5   -33.33683496446       -3.17       -2.31    8.6
  6   -33.33688206839       -4.33       -2.43    1.5
  7   -33.33694386271       -4.21       -3.68    2.8
  8   -33.33694392108       -7.23       -4.04    5.6


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298074570922                   -0.85    5.4
  2   -8.300241634592       -2.66       -1.58    1.0
  3   -8.300447915533       -3.69       -2.54    4.4
  4   -8.300378523348   +   -4.16       -2.29   15.2
  5   -8.300463839678       -4.07       -3.34    4.1
  6   -8.300464497771       -6.18       -3.73    2.2
  7   -8.300464641834       -6.84       -4.63    2.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32716212934                   -0.56    6.9
  2   -33.31943392942   +   -2.11       -1.27    2.0
  3   -17.35984327719   +    1.20       -0.46    8.2
  4   -33.32073669415        1.20       -1.71    6.5
  5   -33.20847352038   +   -0.95       -1.31    4.2
  6   -32.35007059338   +   -0.07       -1.07    4.4
  7   -33.32030319685       -0.01       -1.88    4.6
  8   -33.33511305372       -1.83       -2.33    2.8
  9   -33.33632710206       -2.92       -2.43    3.0
 10   -33.33664698758       -3.50       -2.76    2.0
 11   -33.33680085316       -3.81       -2.94    2.4
 12   -33.33694126193       -3.85       -3.60    3.1
 13   -33.33694307145       -5.74       -3.73    3.8
 14   -33.33694381928       -6.13       -4.23    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.